In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
! pip install boto3
! pip install SentencePiece==0.1.94
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 65.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 69.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 23.4 MB/s eta 0:00:00


In [1]:
%cd /content/drive/MyDrive/transformer-drg-style-transfer-master/


/content/drive/.shortcut-targets-by-id/1gvgdEyQQFFN43xnL2_DdI4rUtMI5gnmU/transformer-drg-style-transfer-master


In [2]:
import torch, os
from pytorch_pretrained_bert import OpenAIGPTTokenizer, OpenAIGPTLMHeadModel
from tqdm import tqdm
import numpy as np
from pytorch_pretrained_bert.file_utils import PYTORCH_PRETRAINED_BERT_CACHE
from pytorch_pretrained_bert.modeling import BertForSequenceClassification, BertConfig, WEIGHTS_NAME, CONFIG_NAME
#from pytorch_pretrained_bert.tokenization import BertTokenizer
from pytorch_pretrained_bert.optimization import BertAdam, warmup_linear
from bertviz.bertviz.pytorch_pretrained_bert import BertModel, BertTokenizer

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [3]:
from tqdm.auto import tqdm

## Delete and Generate

In [4]:
special_tokens = ['<POS>', '<NEG>','<CON_START>','<START>','<END>'] # Set the special tokens
tokenizer = OpenAIGPTTokenizer.from_pretrained('openai-gpt', special_tokens=special_tokens)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = OpenAIGPTLMHeadModel.from_pretrained('openai-gpt', num_special_tokens=len(special_tokens))

In [5]:
#path = os.path.join(os.getcwd(), "./log_yelp_bert_best_head_preprocessed/pytorch_model_zero_grad_1.bin") ## Model Path
path = "/content/drive/MyDrive/transformer-drg-style-transfer-master/t5_tfidf/model_output/pytorch_model_zero_grad_1.bin"
model_state_dict = torch.load(path, map_location=device)
model.load_state_dict(model_state_dict)
model.to(device)
model.eval()

OpenAIGPTLMHeadModel(
  (transformer): OpenAIGPTModel(
    (tokens_embed): Embedding(40483, 768)
    (positions_embed): Embedding(512, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x Block(
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_1): BertLayerNorm()
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): BertLayerNorm()
      )
    )
  )
  (lm_head): OpenAIGPTLMHead(
    (decoder): Linear(in_features=768, out_features=40483, bias=False)
  )
)

In [6]:
from transformers import T5Tokenizer, T5Model, T5ForConditionalGeneration
tokenizer_cls = T5Tokenizer.from_pretrained('t5-base', max_length=70)
model_cls =  T5ForConditionalGeneration.from_pretrained('/content/drive/MyDrive/transformer-drg-style-transfer-master/t5_base_yelp_sentiment/', return_dict = True)
model_cls.load_state_dict(torch.load('/content/drive/MyDrive/transformer-drg-style-transfer-master/t5_base_yelp_sentiment/pytorch_model.bin', map_location=torch.device(device)))
model_cls.eval()
model_cls.to(device)

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [9]:
max_seq_len=70
sm = torch.nn.Softmax(dim=-1)

In [7]:
model.config.n_positions

512

In [8]:
def preditction_with_beam_search(ref_text, beam_width=3, vocab_length=40483):
    """
    This function decodes sentences using Beam Seach. 
    It will output #sentences = beam_width. This function works on a single example.
    
    ref_text : string : Input sentence
    beam_width : int : Width of the output beam
    vocab_length : int : Size of the Vocab after adding the special tokens
    """
    
    done = [False for i in range(beam_width)] # To track which beams are already decoded
    stop_decode = False
    decoded_sentences=[] # List of decoded sentences at any given time
    
    sm = torch.nn.Softmax(dim=-1) # To calculate Softmax over the final layer Logits
    tokens = tokenizer.tokenize(ref_text) # Tokenize the input text
    
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokens) # Convert tokens to ids
    index_tokens = [indexed_tokens for i in range(beam_width)] # Replication of Input ids for all the beams

    #index_tokens = [indexed_tokens for i in range(beam_width)]
    torch_tensor = torch.tensor(index_tokens).to(device)
    beam_indexes = [[] for i in range(beam_width)] # indexes of the current decoded beams
    best_scoes = [0 for i in range(beam_width)] # A list of lists to store Probability values of each decoded token of best beams
    count = 0
    while count < model.config.n_positions and not stop_decode:
        if count == 0: # For the first step when only one sentence is availabe
            with torch.no_grad():
                # Calculate output probability distribution over the Vocab,
                preds = sm(model(torch_tensor)) #  shape = [beam_bidth, len(input_sen)+1,Vocab_length]
            top_v, top_i = preds[:,-1,:].topk(beam_width) # Fatch top indexes and it's values
            [beam_indexes[i].append(top_i[0][i].tolist()) for i in range(beam_width)] # Update the Beam indexes
            # Update the best_scores, for first time just add the topk values directly
            for i in range(beam_width):
                best_scoes[i] = top_v[0][i].item()
            count += 1
        else: # After first step
            # Prepare the current_state by concating original input and decoded beam indexes
            current_state = torch.cat((torch_tensor, torch.tensor(beam_indexes).to(device)), dim=1)
            # Prediction on the current state
            with torch.no_grad():
                preds = sm(model(current_state))
            # Multiply new probability predictions with corresponding best scores
            # Total socres = beam_width * Vocab_Size
            flatten_score = (preds[:,-1,:]*torch.tensor(best_scoes).to(device).unsqueeze(1)).view(-1)
            # Fatch the top scores and indexes 
            vals, inx = flatten_score.topk(beam_width)
            # best_score_inx saves the index of best beams after multiplying the probability of new prediction
            best_scoes_inx = (inx / vocab_length).tolist()
            best_scoes = vals.tolist()
            # Unflatten the index 
            correct_inx = (inx % vocab_length).tolist()
            
            # Check if done for all the Beams
            for i in range(beam_width):
                if correct_inx[i] == tokenizer.special_tokens["<END>"]:
                    done[i] = True
            # Update the best score for each the current Beams
            for i in range(beam_width):
                if not done[i]:
                    best_scoes[i] = vals.tolist()[i]
            # Check is All the Beams are Done
            if (sum(done) == beam_width):
                stop_decode = True
            # Prepapre the new beams
            temp_lt=[0 for i in range(beam_width)]
            for i,x in enumerate(best_scoes_inx):
                temp_lt[i] = beam_indexes[int(x)] + [correct_inx[i]]
            # Update the Beam indexes
            beam_indexes = temp_lt
            del temp_lt
            count += 1
    # Decode All the beam indexes to till <END> token only and convert into sentence
    for i in range(beam_width):
        try:
            end_index = beam_indexes[i].index(tokenizer.special_tokens["<END>"])
        except ValueError:
            end_index = len(beam_indexes[i])
            
        decoded_sentences.append(tokenizer.decode(beam_indexes[i][:end_index]))
        
    return decoded_sentences

In [9]:
def get_best_sentence(input_sentences, sentiment= 0):
    
    """
    This function selects the sentence from the Beam of the sentences,
    based on the classification label.

    input_sentences : list of strings : Sentences generated by the Beam search decoding
    sentiment: int : Expected sentiment (in general class for the classification)
    model_name_or_path: str : HuggingFace model name or path to T5 model
    """

    # Generate input prompt for T5
    prompt = f"classify the sentiment of the following sentences as {sentiment}. sentence:"

    # Create batch inputs for T5 model
    input_ids = []
    input_masks = []
    for sen in input_sentences:
        input_str = f"{prompt} {sen}"
        input_mask = tokenizer_cls.encode_plus(input_str, truncation=True, pad_to_max_length=True,max_length=70, )
        input_masks.append(input_mask['attention_mask'])
        input_ids.append(input_mask['input_ids'])
        #input_ids.append(tokenizer_cls.encode(input_str, truncation=True, max_length=70))
      
    # print(input_ids)
    # print("Input ids shape:", len(input_ids), len(input_ids[0]))
    # print("Input masks shape:", len(input_masks), len(input_ids[0]))

    input_ids = torch.tensor(input_ids).to(model_cls.device)
    #print(input_sentences)
    #input_masks = torch.tensor(input_masks).to(model_cls.device)


    #Predict sentiment labels using T5
    # with torch.no_grad():
    #     outputs = model_cls.generate(input_ids=input_ids, decoder_input_ids = input_ids , max_length=2)



    # # Decode predicted sentiment labels
    # preds = [tokenizer_cls.decode(output, skip_special_tokens=True) for output in outputs]
    # output_sentences = [sen.strip() for sen in preds]

    with torch.no_grad():
        logits = model_cls(input_ids=input_ids, decoder_input_ids=input_ids)
        logits = logits.logits[:, 1]
        softmaxed_logits = torch.softmax(logits, dim=-1)

    scores = softmaxed_logits
    preds = scores.tolist()
    inx, inx_val = None, 0
    for i in range(len(input_sentences)):
        temp = preds[i][1-sentiment]
        if temp > inx_val:
            inx = i
            inx_val = temp
    return input_sentences[inx]
    #Find the sentence with the highest sentiment score
    best_sentence_index = max(range(len(output_sentences)), key=lambda i: scores[i])


    return input_sentences[inx]


## Example

In [12]:
op = preditction_with_beam_search("<POS> <CON_START> it is not terrible , but it is very good . <START>",4)
op

['it is not terrible , but it is very good .',
 'it is not terrible , but it is very good .',
 'it is not terrible , but it is very good .',
 'it is not terrible , but it is very good .']

In [13]:
get_best_sentence(op)

Input ids shape: 4 26
Input masks shape: 4 26


Input length of decoder_input_ids is 26, but `max_length` is set to 2. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


'it is not terrible , but it is very good .'

### Predictions for the reference files

In [11]:
output_dir = "/content/drive/MyDrive/transformer-drg-style-transfer-master/outputs_drg_yelp"
os.listdir(output_dir)

['reference_1_predictions_with_full_sentence_match_beam_search_bm5.txt',
 'reference_0_predictions_with_full_sentence_match_beam_search_bm5.txt',
 'reference_1_predictions_with_beam_search.txt',
 'reference_1.txt',
 'reference_0.txt',
 'reference_0_predictions_with_beam_search.txt']

In [12]:
reference_0 = '/content/drive/MyDrive/transformer-drg-style-transfer-master/outputs_drg_yelp/reference_0.txt'
reference_1 = '/content/drive/MyDrive/transformer-drg-style-transfer-master/outputs_drg_yelp/reference_1.txt'

In [ ]:
with open(os.path.join(output_dir,"reference_0_predictions_with_beam_search.txt") ,"w+", encoding='utf-8') as out_fp:
    c = 0
    with open(reference_0, 'r') as fp:
        for line in tqdm(fp):
            out_sen = preditction_with_beam_search(line.strip(), beam_width=5, vocab_length=max(tokenizer.special_tokens.values()) + 1)
            #print(out_sen)
            print(c,get_best_sentence(out_sen, sentiment = 1))
            c += 1
            out_fp.write(get_best_sentence(out_sen, sentiment = 1) + "\n")

In [ ]:
with open(os.path.join(output_dir,"reference_1_predictions_with_beam_search.txt") ,"w+", encoding='utf-8') as out_fp:
    c = 0
    with open(reference_1, 'r') as fp:
        for line in tqdm(fp):
            out_sen = preditction_with_beam_search(line.strip(), beam_width=5, vocab_length = max(tokenizer.special_tokens.values()) + 1)
            print(c,get_best_sentence(out_sen, sentiment = 0))
            c += 1
            out_fp.write(get_best_sentence(out_sen, sentiment = 0) + "\n")

## Delete, Retrieve and Generate

In [ ]:
special_tokens = ['<ATTR_WORDS>','<CON_START>','<START>','<END>'] # Set the special tokens
tokenizer = OpenAIGPTTokenizer.from_pretrained('openai-gpt', special_tokens=special_tokens)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = OpenAIGPTLMHeadModel.from_pretrained('openai-gpt', num_special_tokens=len(special_tokens))

In [ ]:
#path = os.path.join(os.getcwd(), "./log_yelp_retireve_edit_bert_best_head_preprocessing/pytorch_model_zero_grad_1.bin")
path = "/content/drive/MyDrive/transformer-drg-style-transfer-master (1)/model_output/drg_output/pytorch_model_zero_grad_1.bin"
model_state_dict = torch.load(path, map_location=device)
model.load_state_dict(model_state_dict)
model.to(device)
model.eval()

In [ ]:
# Output dir have reference files generated using TFIDF for retrieve attributes from opposite corpus
output_dir = '/content/drive/MyDrive/transformer-drg-style-transfer-master (1)/data/yelp/processed_files_with_bert_with_best_head/delete_retrieve_edit_model/tfidf'
os.listdir(output_dir)

['reference_1.txt',
 'reference_0_predictions_with_full_sentence_match_beam_search_bm5.txt',
 'reference_1_predictions_with_full_sentence_match_beam_search_bm5.txt']

In [ ]:
with open(os.path.join(output_dir,"reference_0_predictions_with_full_sentence_match_beam_search_bm5.txt") ,"w", encoding='utf-8') as out_fp:
    c = 0
    with open() as fp:
        for line in fp:
            out_sen = preditction_with_beam_search(line.strip(), beam_width=5, vocab_length=max(tokenizer.special_tokens.values()) + 1)
            print(c,get_best_sentence(out_sen, sentiment = 1))
            c += 1
            out_fp.write(get_best_sentence(out_sen, sentiment = 1) + "\n")

In [ ]:
# Output dir have reference files generated using TFIDF for retrieve attributes from opposite corpus
output_dir = '/content/drive/MyDrive/transformer-drg-style-transfer-master (1)/outputs_drg_yelp'
os.listdir(output_dir)

[]

In [16]:
with open(os.path.join(output_dir,"reference_1_predictions_with_full_sentence_match_beam_search_bm5.txt") ,"w", encoding='utf-8') as out_fp:
    c = 0
    with open('/content/drive/MyDrive/transformer-drg-style-transfer-master (1)/data/yelp/processed_files_with_bert_with_best_head/delete_retrieve_edit_model/tfidf/reference_1.txt') as fp:
        for line in fp:
            out_sen = preditction_with_beam_search(line.strip(), beam_width=5, vocab_length=max(tokenizer.special_tokens.values()) + 1)
            print(c,get_best_sentence(out_sen, sentiment = 0))
            c += 1
            out_fp.write(get_best_sentence(out_sen, sentiment = 0) + "\n")

FileNotFoundError: ignored